In [1]:
import os
import yaml
from astropy.table import Table
import matplotlib.pyplot as plt
import healpy as hp
import numpy as np
import fitsio
import datetime
%matplotlib inline

In [2]:
import desisim
import desisurvey
import desimodel.io
import desimodel.footprint

# Environment Variables 

Set some environment variables and create output directories



In [15]:
alldirs = {}

os.environ['DESISURVEY_OUTPUT'] = '/global/projecta/projectdirs/desi/datachallenge/surveysim2017/baseline_1m'

basedir = os.path.join(os.environ['SCRATCH'],'quicksurvey_example')

surveydir = os.path.join(basedir,'survey')
fiberassigndir =  os.path.join(basedir, 'fiberassign')
alldirs['surveydir'] = surveydir
alldirs['fiberassigndir'] = fiberassigndir

brighttargetdir = os.path.join(basedir,'targets/no_spectra/bright')
darktargetdir = os.path.join(basedir,'targets/no_spectra/dark')
alldirs['brighttargetdir'] = brighttargetdir
alldirs['darktargetdir'] = darktargetdir

zcatbrightdir = os.path.join(basedir,'zcat/bright')
zcatdarkdir = os.path.join(basedir,'zcat/dark')
alldirs['zcatbrightdir'] = zcatbrightdir
alldirs['zcatdarkdir'] = zcatdarkdir

alltargetdir = os.path.join(basedir,'targets/no_spectra/bright')
zcatalldir = os.path.join(basedir,'zcat/all')
alldirs['alltargetdir'] = alltargetdir
alldirs['zcatalldir'] = zcatalldir



In [16]:
for k in alldirs.keys():
    print('Creating {}: {}'.format(k, alldirs[k]))
    os.makedirs(alldirs[k], exist_ok=True)

Creating alltargetdir: /global/cscratch1/sd/forero/quicksurvey_example/targets/no_spectra/bright
Creating darktargetdir: /global/cscratch1/sd/forero/quicksurvey_example/targets/no_spectra/dark
Creating fiberassigndir: /global/cscratch1/sd/forero/quicksurvey_example/fiberassign
Creating brighttargetdir: /global/cscratch1/sd/forero/quicksurvey_example/targets/no_spectra/bright
Creating zcatdarkdir: /global/cscratch1/sd/forero/quicksurvey_example/zcat/dark
Creating zcatalldir: /global/cscratch1/sd/forero/quicksurvey_example/zcat/all
Creating zcatbrightdir: /global/cscratch1/sd/forero/quicksurvey_example/zcat/bright
Creating surveydir: /global/cscratch1/sd/forero/quicksurvey_example/survey


# Compute exposures files from `progress` data

`progress.fits` is the file that summarizes the survey simulation performed with `desisurvey`

We read that file and write to disk a file with all the `exposures`
made during the survey and two more files with the exposures for dark and bright
surveys. **NOTE**: We include grey exposures into `all_exposures_dark` file.

Bright twilight tiles should be excluded because we don't have a right sky model, but we are including them here anyway.

In [25]:
from desisurvey.progress import Progress

import desisurvey.config
config = desisurvey.config.Configuration()
config.first_day()

datetime.date(2019, 12, 1)

In [27]:
progress_filename = os.path.join(os.environ['DESISURVEY_OUTPUT'],'progress.fits')
exposures_filename = os.path.join(alldirs['surveydir'],'exposures.fits')
Progress(restore=progress_filename).get_exposures().write(exposures_filename, overwrite=True)
    
explist = Table.read(exposures_filename)

# separate the exposures for dark and bright programs
isbright = explist['PASS'] > 4 
isdark = ~isbright
    
exposurefile_bright = os.path.join(alldirs['surveydir'],'all_exposures_bright.fits')
Table(explist[isbright]).write(exposurefile_bright, overwrite=True)

exposurefile_dark = os.path.join(alldirs['surveydir'],'all_exposures_dark.fits')
Table(explist[~isbright]).write(exposurefile_dark, overwrite=True)


INFO:progress.py:145:__init__: Loaded progress from /global/projecta/projectdirs/desi/datachallenge/surveysim2017/baseline_1m/progress.fits.


In [28]:
# Make a simple plot
plt.plot(explist['ra'][isdark], explist['dec'][isdark], 'kx', alpha=0.8, label='dark')
plt.plot(explist['ra'][isgray], explist['dec'][isgray], 'k.', alpha=0.2, label='gray')
plt.plot(explist['ra'][isbright], explist['dec'][isbright], 'mx', alpha=0.8, label='bright')
plt.legend(loc='upper right')

KeyError: 'ra'

# Select a subset of tiles

We pick 20 tiles nearest RA=180, dec=18. That are already included in the survey simulation outputs.
`fiberassign` will run on those tiles.

In [ ]:
tiles = desimodel.io.load_tiles()
explist = Table.read(os.path.join(alldirs['surveydir'],'exposures.fits').replace('$SCRATCH',os.environ['SCRATCH']))

In [ ]:
ntiles = 20

xtiles = tiles[np.in1d(tiles['TILEID'], explist['tileid'])]
xyz = hp.ang2vec(xtiles['ra'], xtiles['dec'], lonlat=True)
center = hp.ang2vec(180, 18, lonlat=True)
d2 = ((xyz - center)**2).sum(axis=1)
ii = np.argsort(d2)[0:ntiles]
xtiles = xtiles[ii]

nexp = np.count_nonzero(np.in1d(explist['tileid'], xtiles['TILEID']))
print('{} tiles covered by {} exposures'.format(len(xtiles), nexp))

In [ ]:
#- row indices of exposures of the selected tiles
iobs = np.where(np.in1d(explist['tileid'], xtiles['TILEID']))[0]
assert np.all(np.in1d(explist['tileid'][iobs], xtiles['TILEID']))
print(iobs)

# The exposure list could have a repeated tileid. That would mean that 
# the tile had to be targetted more than once.
explist[iobs]

In [ ]:
# save the reduce list of exposures for bright and dark/gray survey
mini_explist = explist[iobs]
isbright = mini_explist['pass'] > 4

exposurefile_bright = os.path.join(surveydir,'mini_exposures_bright.fits')
Table(mini_explist[isbright]).write(exposurefile_bright, overwrite=True)


exposurefile_dark = os.path.join(surveydir,'mini_exposures_dark.fits')
Table(mini_explist[~isbright]).write(exposurefile_dark, overwrite=True)


exposurefile = os.path.join(surveydir,'mini_exposures.fits')
Table(mini_explist).write(exposurefile, overwrite=True)

In [ ]:
Table(mini_explist[isbright])

In [ ]:
Table(mini_explist[~isbright])

Plot the tiles and pixels

In [ ]:
def plot_tile(ra, dec, r=1.606, color='k'):
    '''Approximate plot of tile location'''
    ang = np.linspace(0, 2*np.pi, 100)
    x = ra + r*np.cos(ang)/np.cos(np.radians(dec))
    y = dec + r*np.sin(ang)
    plt.plot(x,y, '-', color=color)


plt.figure(figsize=(12,3))
plt.subplot(121)
plt.plot(tiles['RA'], tiles['DEC'], 'k,', alpha=0.5)
plt.plot(explist['ra'], explist['dec'], 'b.', alpha=0.5)
plt.plot(xtiles['RA'], xtiles['DEC'], 'rx')
plt.xlim(0,360); plt.ylim(-20, 80)

plt.xlim(175, 185); plt.ylim(13, 23)

In [ ]:
#- Write subset of tiles table to file for input to select_mock_targets --no-spectra
tilefile = os.path.join(alldirs['darktargetdir'],'test-tiles.fits')
Table(xtiles).write(tilefile, overwrite=True)

tilefile = os.path.join(alldirs['brighttargetdir'],'test-tiles.fits')
Table(xtiles).write(tilefile, overwrite=True)
Table(xtiles)

# Run serial select_mock_targets --no-spectra

In [ ]:
select_mock_targets_cmd  = "select_mock_targets --no-spectra -c {yamlconfigfile} --nside {healpixnside} --output_dir {outputdir} --seed {seed} --tiles {tilefile}"

cmd = select_mock_targets_cmd.format(yamlconfigfile=os.path.join(alldirs['darktargetdir'],'select-mock-targets-dark.yaml'),
                                    healpixnside=16,
                                    outputdir=darktargetdir,
                                    seed=10,
                                    tilefile=os.path.join(alldirs['darktargetdir'],'test-tiles.fits'))
print('# Copy and paste this command into a login terminal:\n')
print(cmd)

In [ ]:
cmd = "join_mock_targets --force --mockdir "+darktargetdir
print('# Copy and paste this command into a login terminal:\n')
print(cmd)

In [ ]:
# some sanity checks on the results

In [ ]:
targets = fitsio.read(os.path.join(alldirs['darktargetdir'],'targets.fits'))
truth   = fitsio.read(os.path.join(alldirs['darktargetdir'],'truth.fits'))
mtl     = fitsio.read(os.path.join(alldirs['darktargetdir'],'mtl.fits'))
std     = fitsio.read(os.path.join(alldirs['darktargetdir'],'standards-dark.fits'))
sky     = fitsio.read(os.path.join(alldirs['darktargetdir'],'sky.fits'))

In [ ]:
assert len(truth) == len(targets)                             #- same number of targets and truth
assert np.all(targets['TARGETID'] == truth['TARGETID'])       #- targets and truth are row matched
assert len(targets) == len(np.unique(targets['TARGETID']))    #- no repeated TARGETIDs
assert len(sky) == len(np.unique(sky['TARGETID']))            #- no repeated sky TARGETIDs
assert len(std) == len(np.unique(std['TARGETID']))            #- no repeated std TARGETIDs

assert np.all(np.in1d(targets['TARGETID'], mtl['TARGETID']))  #- all targets are in MTL

#- no sky targets should be in science targets, though it is possible for standards to also be MWS targets
assert not np.any(np.in1d(sky['TARGETID'], targets['TARGETID']))

In [ ]:
plt.figure(figsize=(10,8))
plt.scatter(mtl['RA'], mtl['DEC'], color='blue', alpha=0.1, s=0.01)
plt.plot(std['RA'], std['DEC'], 'm.')

# Run quicksurvey


## Fiberassign dates

From the `surveysim` data we have to find the dates when `fiberassign` is expected to be run

In [ ]:
def compute_fiberassign_dates(exposurefile, outputfile):
    """
    Computes the dates to read fiberassign from the data in a 'exposures' files.
    
    Args:
        exposurefile (str):
        Name of the file with 'exposures' data
    Return:
        fiberassign_dates_run (list):
        List of string dates where fiberassign should be run.
    """
    desisurvey_path = os.getenv('DESISURVEY_OUTPUT')
    exposures = Table.read(exposurefile)
    expdates = []
    for n in exposures['night']:
        a = datetime.datetime.strptime(n, "%Y-%m-%d")
        expdates.append(a.date())
    expdates = np.array(expdates)

    # compute the dates for fiberassign
    progress = Table.read(os.path.join(desisurvey_path,'progress.fits'))
    fiberassign_dates = np.sort(list(set(progress['available'])))
    fiberassign_dates = fiberassign_dates[fiberassign_dates>0]
    
    #load the first date of the survey
    survey_config = os.path.join(desisurvey_path,'config.yaml')
    with open(survey_config, 'r') as pfile:
        params = yaml.load(pfile)
    pfile.close()
    first_day = params['first_day']
    last_day = params['last_day']
 #   print(first_day, last_day)
    
    #compute the dates to run fiberassign to be sure that there is at least one exposure there
    fiberassign_dates_run = []
    one_day = datetime.timedelta(days=1)
    initial_day = first_day 
    for d in fiberassign_dates:
        final_day = first_day + d * one_day
        ii = (expdates > initial_day) & (expdates<=final_day)
        n_in = np.count_nonzero(ii)
        if n_in>0:
#            print(initial_day, final_day, n_in)
            fiberassign_dates_run.append(initial_day.strftime("%Y-%m-%d"))
            initial_day = final_day
    f = open(outputfile, 'w')
    for d in fiberassign_dates_run:
        f.write(d+"\n")
    f.close()

In [ ]:
fiberassignfile_dates_dark = os.path.join(alldirs['fiberassigndir'], 'fiberassign_dates_dark.txt')
exposurefile_bright = exposurefile_dark.replace('$SCRATCH',os.environ['SCRATCH'])
fiberassignfile_dates_dark = fiberassignfile_dates_dark.replace('$SCRATCH',os.environ['SCRATCH'])
compute_fiberassign_dates(exposurefile_dark, fiberassignfile_dates_dark)

In [ ]:
! cat $fiberassignfile_dates_dark

In [ ]:
quicksurvey_cmd = "quicksurvey -T {targetdir} -E {exposures}  --output_dir {outputdir} -f $(which fiberassign)"
quicksurvey_cmd += " -t {fiberassign_template} -D {fiberassign_dates}"

cmd = quicksurvey_cmd.format(targetdir=alldirs['darktargetdir'],
                            exposures=alldirs['surveydir']+'/mini_exposures_dark.fits', 
                            outputdir=alldirs['zcatdarkdir'], 
                            fiberassign_template=alldirs['fiberassigndir']+'/template_fiberassign_dark.txt', 
                            fiberassign_dates=alldirs['fiberassigndir']+'/fiberassign_dates_dark.txt')
print('# Copy and paste this command into a login terminal:\n')
print(cmd)

# Read the final redshift catalog

Read the final redshift catalog.

Blue: input catalog.
Red: objects with a redshift.

In [ ]:
zcatfilename = os.path.join(alldirs['zcatdarkdir'],'3/zcat.fits')
zcat     = fitsio.read(zcatfilename)
zcat_in_mtl = np.in1d(mtl['TARGETID'], zcat['TARGETID'])
plt.figure(figsize=(10,8))
plt.scatter(mtl['RA'], mtl['DEC'], c='b', alpha=0.1, s=0.01)
plt.scatter(mtl[zcat_in_mtl]['RA'], mtl[zcat_in_mtl]['DEC'], c='r', alpha=0.4, s=0.05)
print(np.count_nonzero(zcat_in_mtl))

In [ ]:
a = plt.hist(zcat['Z'], bins=np.linspace(0.0,3.5,20))